Integrantes  
Bryann Alfaro  
Raul Jimenez  
Donaldo Garcia

## Paso 1 - Exploracion de datos

In [3]:
#Load dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv('dataset_pishing.csv')

#Mostrar 5 observaciones
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [4]:
#Mostrar cantidad de observaciones de columna status
df['status'].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

Se puede observar que el dataset esta balanceado por medio de la columna status

## Derivacion de caracteristicas

f1 

In [7]:
#Obtener len de las url
df['len_url'] = df['url'].apply(len)
#show url and len url 
df[['url','len_url']].head()


,url,len_url
0,http://www.crestonwood.com/router.php,37
1,http://shadetreetechnology.com/V4/validation/a...,77
2,https://support-appleld.com.secureupdate.duila...,126
3,http://rgipt.ac.in,18
4,http://www.iracing.com/tracks/gateway-motorspo...,55


f2

In [14]:
from urllib.parse import urlparse
def get_hostname(url):
    return urlparse(url).hostname

df['hostname_len'] = df['url'].apply(get_hostname).apply(len)
df[['url','hostname_len']].head()

,url,hostname_len
0,http://www.crestonwood.com/router.php,19
1,http://shadetreetechnology.com/V4/validation/a...,23
2,https://support-appleld.com.secureupdate.duila...,50
3,http://rgipt.ac.in,11
4,http://www.iracing.com/tracks/gateway-motorspo...,15


f3

In [16]:
#Show ip column
df[['url','ip']].head()

,url,ip
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,1
2,https://support-appleld.com.secureupdate.duila...,1
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0


f4 - f20

In [34]:
#Count special characters
def count_special_characters(url):
    special_characters = ['.','-','@','|','_','~','%','/','?','&','=','*',':',',',';','$','%20']
    count = 0
    #Count every special character in special characters and create every column
    for special_character in special_characters:
        count += url.count(special_character)
        df[f'{special_character}'] = count

df.columns
    

Index(['url', 'ip', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path',
       'punycode', 'port', 'tld_in_path', 'tld_in_subdomain',
       'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain',
       'shortening_service', 'path_extension', 'nb_redirection',
       'nb_external_redirection', 'length_words_raw', 'char_repeat',
       'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
       'longest_words_raw', 'longest_word_host', 'longest_word_path',
       'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints',
       'domain_in_brand', 'brand_in_subdomain', 'brand_in_path',
       'suspecious_tld', 'statistical_report', 'nb_hyperlinks',
       'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks',
       'nb_extCSS', 'ratio_intRedirection', 'ratio_extRedirection',
       'ratio_intErrors', 'ratio_extErrors', 'login_form', 'external_favicon',
       'links_in_tags', 'submit_email', 'ratio_intMedia', 'ratio_extMedia',
       'sf

f25

In [21]:
df.columns.sort_values()
df[['http_in_path','status','url']].head()

,http_in_path,status,url
0,0,legitimate,http://www.crestonwood.com/router.php
1,0,phishing,http://shadetreetechnology.com/V4/validation/a...
2,0,phishing,https://support-appleld.com.secureupdate.duila...
3,0,legitimate,http://rgipt.ac.in
4,0,legitimate,http://www.iracing.com/tracks/gateway-motorspo...


In [24]:
#count https
def count_https(url):
    return 1 if urlparse(url).scheme == 'https' else 0



df['https'] = df['url'].apply(count_https)
df[['url','https']].head()

,url,https
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,0
2,https://support-appleld.com.secureupdate.duila...,1
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0


f26

In [30]:
#count digits in url
def count_digits(url):
    count = 0
    for character in url:
        if character.isdigit():
            count += 1
    return count

df['url_digits'] = df['url'].apply(count_digits)
df[['url','url_digits']].head()

,url,url_digits
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,17
2,https://support-appleld.com.secureupdate.duila...,19
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0


f27

In [29]:
#Count digits in hostname
def count_digits_hostname(url):
    return count_digits(get_hostname(url))

df['digits_hostname'] = df['url'].apply(count_digits_hostname)
df[['url','digits_hostname']].head()

,url,digits_hostname
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,0
2,https://support-appleld.com.secureupdate.duila...,0
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0
